In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import googlemaps

In [2]:
# API from NM work gmail
gmaps = googlemaps.Client(key=[ENTER YOUR API HERE])

In [38]:
efap = pd.read_csv('/home/nick/committees/SOC_HealthyNYC_2022/food_access/data/EFAP_pdf_4_7_22.csv', dtype=str) 
efap.head()

,ID,TYPE,PROGRAM,PHONE,DISTADD,DISTBORO,DISTZIP,DAYS
0,84948,FP,ACTS COMMUNITY DEVELOPMENT CORPORATION,718 415-1170,2114 MERMAID AVE,BK,11224,TUE/THUR 6-7PM/11AM-12PM
1,84664,SK,ACTS COMMUNITY DEVELOPMENT CORPORATION,718-415-1170,2114 MERMAID AVE,BK,11224,"WED,FRI 6:30-7:30PM"
2,85247,FP,ADMIRABLE CHURCH INC,347-779-7192,1910 FLATBUSH AVENUE,BK,11210,SAT 10AM-12PM
3,85537,FP,APNA FOOD PANTRY BRIGHTON BEACH,718-513-4700,225 NEPTUNE AVENUE,BK,11235,WED 2-4PM
4,85330,FP,BAY VIEW HOUSES RESIDENT ASSOCIATION,347-371-9794,2085 ROCKAWAY PARKWAY,BK,11236,THUR 10AM-2PM


In [39]:
conditions = [
    (efap["DISTBORO"]=='BK'),
    (efap["DISTBORO"]=='BX'),
    (efap["DISTBORO"]=='NY'),
    (efap["DISTBORO"]=='QN'),
    (efap["DISTBORO"]=='SI'),
]
choices = ["Brooklyn", "Bronx", "Manhattan", "Queens", "Staten Island"]
efap["boro"] = np.select(conditions, choices)

efap['Address'] = efap['PROGRAM'] + ", " + efap['DISTADD'] + ", " + efap['boro'] + ", " + efap["DISTZIP"] + ", NY"

efap.head()

,ID,TYPE,PROGRAM,PHONE,DISTADD,DISTBORO,DISTZIP,DAYS,boro,Address
0,84948,FP,ACTS COMMUNITY DEVELOPMENT CORPORATION,718 415-1170,2114 MERMAID AVE,BK,11224,TUE/THUR 6-7PM/11AM-12PM,Brooklyn,"ACTS COMMUNITY DEVELOPMENT CORPORATION , 2114 ..."
1,84664,SK,ACTS COMMUNITY DEVELOPMENT CORPORATION,718-415-1170,2114 MERMAID AVE,BK,11224,"WED,FRI 6:30-7:30PM",Brooklyn,"ACTS COMMUNITY DEVELOPMENT CORPORATION , 2114 ..."
2,85247,FP,ADMIRABLE CHURCH INC,347-779-7192,1910 FLATBUSH AVENUE,BK,11210,SAT 10AM-12PM,Brooklyn,"ADMIRABLE CHURCH INC , 1910 FLATBUSH AVENUE , ..."
3,85537,FP,APNA FOOD PANTRY BRIGHTON BEACH,718-513-4700,225 NEPTUNE AVENUE,BK,11235,WED 2-4PM,Brooklyn,"APNA FOOD PANTRY BRIGHTON BEACH , 225 NEPTUNE ..."
4,85330,FP,BAY VIEW HOUSES RESIDENT ASSOCIATION,347-371-9794,2085 ROCKAWAY PARKWAY,BK,11236,THUR 10AM-2PM,Brooklyn,"BAY VIEW HOUSES RESIDENT ASSOCIATION , 2085 RO..."


In [40]:
addresses = efap['Address'].unique()
len(addresses)

468

In [41]:
lat=[]
lng=[]
# loop through each address to search
for i, address in enumerate(addresses):
    #use the google maps interface to search for each address
    geocode_result = gmaps.geocode(address)
    #extract latitude & longitude and add to list of results
    if geocode_result:
        lat.append(geocode_result[0]['geometry']['location']['lat'])
        lng.append(geocode_result[0]['geometry']['location']['lng'])
    else: 
        lat.append('NA')
        lng.append('NA')

In [42]:
# create a table of results (address, lat, lon)
df_locations = pd.DataFrame(addresses).rename(columns = {0:'Address'})

df_locations['lat'] = lat
df_locations['lon'] = lng

df_locations

,Address,lat,lon
0,"ACTS COMMUNITY DEVELOPMENT CORPORATION , 2114 ...",40.575991,-73.988398
1,"ADMIRABLE CHURCH INC , 1910 FLATBUSH AVENUE , ...",40.623102,-73.937755
2,"APNA FOOD PANTRY BRIGHTON BEACH , 225 NEPTUNE ...",40.581846,-73.961627
3,"BAY VIEW HOUSES RESIDENT ASSOCIATION , 2085 RO...",40.633049,-73.886791
4,"BEDFORD CENTRAL PRESBYTERIAN CHURCH , 1200 DEA...",40.676611,-73.949720
...,...,...,...
463,"ST. EDWARD FOOD PANTRY , 6581 HYLAN BOULEVARD ...",40.510309,-74.219768
464,"STAPLETON U.A.M.E. CHURCH , 49 TOMPKINS AVE , ...",40.623399,-74.078635
465,"STAPLETON U.A.M.E. CHURCH , 49 TOMPKINS AVENUE...",40.623399,-74.078635
466,"THE SALVATION ARMY STAPLETON CORPS , 15 BROAD ...",40.625814,-74.075731


In [43]:
df_locations.to_csv('/home/nick/committees/SOC_HealthyNYC_2022/food_access/data/address_lat_lon.csv', index=False)

In [46]:
efap_merged = efap.merge(df_locations, how='left', on='Address')
# remove full address columns
efap_merged = efap_merged.loc[:, efap_merged.columns != "Address"]

efap_merged

,ID,TYPE,PROGRAM,PHONE,DISTADD,DISTBORO,DISTZIP,DAYS,boro,lat,lon
0,84948,FP,ACTS COMMUNITY DEVELOPMENT CORPORATION,718 415-1170,2114 MERMAID AVE,BK,11224,TUE/THUR 6-7PM/11AM-12PM,Brooklyn,40.575991,-73.988398
1,84664,SK,ACTS COMMUNITY DEVELOPMENT CORPORATION,718-415-1170,2114 MERMAID AVE,BK,11224,"WED,FRI 6:30-7:30PM",Brooklyn,40.575991,-73.988398
2,85247,FP,ADMIRABLE CHURCH INC,347-779-7192,1910 FLATBUSH AVENUE,BK,11210,SAT 10AM-12PM,Brooklyn,40.623102,-73.937755
3,85537,FP,APNA FOOD PANTRY BRIGHTON BEACH,718-513-4700,225 NEPTUNE AVENUE,BK,11235,WED 2-4PM,Brooklyn,40.581846,-73.961627
4,85330,FP,BAY VIEW HOUSES RESIDENT ASSOCIATION,347-371-9794,2085 ROCKAWAY PARKWAY,BK,11236,THUR 10AM-2PM,Brooklyn,40.633049,-73.886791
...,...,...,...,...,...,...,...,...,...,...,...
500,82139,FP,STAPLETON U.A.M.E. CHURCH,718 273-2857,49 TOMPKINS AVE,SI,10304,"TUE,FRI 10AM-1PM",Staten Island,40.623399,-74.078635
501,81360,SK,STAPLETON U.A.M.E. CHURCH,718 273-2857,49 TOMPKINS AVENUE,SI,10304,MON/WED 3-4PM/12-1PM,Staten Island,40.623399,-74.078635
502,81039,FP,THE SALVATION ARMY STAPLETON CORPS,718 448-8480,15 BROAD STREET,SI,10304,TUE-THUR 12:30-2:30PM,Staten Island,40.625814,-74.075731
503,81040,SK,THE SALVATION ARMY STAPLETON CORPS,718 448-8480,15 BROAD STREET,SI,10304,"MON,FRI 11:30AM-12:30PM",Staten Island,40.625814,-74.075731


In [ ]:
efap_merged.to_csv('/home/nick/committees/SOC_HealthyNYC_2022/food_access/data/efap_geo.csv', index=False)